In [2]:
import os
import re


BASE_DATA_DIR = os.path.join(os.getcwd(), 'data\\SpamSMS') # датасет лежит в data
subdirs = ['ham', 'spam']
data_file_name = 'SMSSpamCollection.txt'

Преобразуйте базу следующим образом: пусть будет две папки spam и ham. В папке spam - спам-сообщения, в папке ham,
соответственно, обычные.

In [10]:
# создаём /ham и /spam
for _, subdir in enumerate(subdirs):
    subdir_path = os.path.join(BASE_DATA_DIR, subdir)
    os.makedirs(subdir_path, exist_ok=True)

# читаем сообщения с игнорированием ошибок, так как с юникодом что то внутри не так
with open(os.path.join(BASE_DATA_DIR, data_file_name), 'r', errors='ignore') as f:
    for i, line in enumerate(f):
        format_line = f"{' '.join(line.split()[1:])}" # убираем в строке маркер ham|spam
        if line.split()[0] == 'ham':
            file_path = os.path.join(BASE_DATA_DIR, subdirs[0], f'{i}.txt')
            with open(file_path, 'a') as f_txt:
                f_txt.write(format_line)
        elif line.split()[0] == 'spam':
            file_path = os.path.join(BASE_DATA_DIR, subdirs[1], f'{i}.txt')
            with open(file_path, 'a') as f_txt:
                f_txt.write(format_line)

1. Определите общее количество сообщений, количество спам и обычных сообщений.

In [11]:
total_sms = 0
for _, subdir in enumerate(subdirs):
    subdir_path = os.path.join(BASE_DATA_DIR, subdir)
    files = os.listdir(subdir_path)
    total_sms += len(files)
    print(f'{subdir} SMS: {len(files)}')
print(f'total SMS: {total_sms}')

ham SMS: 4827
spam SMS: 747
total SMS: 5574


**Ответ**:

ham SMS: 4827

spam SMS: 747

total SMS: 5574

2. Найдите минимальную, максимальную и среднюю длину файлов для всех сообщений и отдельно для каждой
из категорий (спам и обычные).

In [19]:
# храним длины в списках, а списки в словаре
length_file_info = {'spam': list(),
                    'ham': list()}

for _, subdir in enumerate(subdirs):
    # получаем все пути
    subdir_path = os.path.join(BASE_DATA_DIR, subdir)
    files = os.listdir(subdir_path)

    # проходим по файлам и получаем информацию о размере файла в байтах через метод
    # os.stat() и атрибут .st_size
    for _, file in enumerate(files):
        file_info = os.stat(os.path.join(subdir_path, file))
        length_file_info[subdir].append(file_info.st_size)

    # вывод информации, средний размер округлен до 2-ух знаков после запятой
    print(f"\n\n{subdir} (bytes):\nmin length: {min(length_file_info[subdir])}\nmax length: {max(length_file_info[subdir])}\naverage:\
     {round(sum(length_file_info[subdir]) / len(length_file_info[subdir]))}")

# соединяем два списка в один для вывода общей информации
total_info = length_file_info['spam'].copy()
total_info.extend(length_file_info['ham'])
print(f"\n\nTOTAL (bytes):\nmin length: {min(total_info)}\nmax length: {max(total_info)}\naverage:\
 {round(sum(total_info) / len(total_info))}")



ham (bytes):
min length: 2
max length: 910
average:     71


spam (bytes):
min length: 13
max length: 224
average:     139


TOTAL (bytes):
min length: 2
max length: 910
average: 80


**Ответ**:

ham (bytes):

    min length: 2
    max length: 910
    average:     71


spam (bytes):

    min length: 13
    max length: 224
    average:     139


TOTAL (bytes):

    min length: 2
    max length: 910
    average: 80

3. Найдите 20 самых популярных слов по всем сообщениям и по каждой из категорий. Словом будем считать любой набор букв,
отделенный с двух сторон пробелами либо другими разделительными символами - точкой, запятой, восклицательным знаком,
и т.д. Или отделенный с одной стороны, если этот набор находится в начале/конце сообщения.

In [4]:
# Словарь хранится в словаре, где ключом является само слово, а значением по ключу -
# количество вхождений слова
dictionary = {'spam': dict(),
              'ham': dict(),
              'total': dict()}

for _, subdir in enumerate(subdirs):
    subdir_path = os.path.join(BASE_DATA_DIR, subdir)
    files = os.listdir(subdir_path)

    for _, file in enumerate(files):
        file_path = os.path.join(subdir_path, file)# errors='ignore'
        with open(file_path, 'r',errors='ignore' ) as f:
            # Считываем строку и выделяем из неё слова: регфильтр настроен на то, чтобы считать словом
            # последовательность букв, в которой может содеражаться апостроф или дефис, но не могут цифры
            words = re.findall(r"\b[A-Za-z-']+\b", f.readline())
            for word in words:
                # Проверяем, пуст ли словарь, если да - инициализируем пару (слово(ключ), 1).
                # Проверка нужна, так как попытка получить ключи у пустого словаря вызовет ошибку
                if not bool(dictionary[subdir]):
                    dictionary[subdir][word] = 1

                elif word in dictionary[subdir].keys():
                    dictionary[subdir][word] += 1
                else:
                    dictionary[subdir][word] = 1

# Объединяем два словаря в словарь 'total'
dictionary['total'] = {**dictionary['spam'], **dictionary['ham']}
for key, value in dictionary['total'].items():
    if key in dictionary['spam'] and key in dictionary['ham']:
        dictionary['total'][key] = value + dictionary['spam'][key]

# Отсортируем по второму значению в паре словаря, то есть по количеству вхождений
sorted_total = sorted(dictionary['total'].items(), key=lambda x: x[1], reverse=True)
sorted_spam = sorted(dictionary['spam'].items(), key=lambda x: x[1], reverse=True)
sorted_ham = sorted(dictionary['ham'].items(), key=lambda x: x[1], reverse=True)

# Вывод информации со срезом [:20], то есть топ-20
print(f'20 top total words: {sorted_total[:20]}')
print(f'20 top spam words: {sorted_spam[:20]}')
print(f'20 top ham words: {sorted_ham[:20]}')

20 top total words: [('to', 2159), ('you', 1832), ('I', 1533), ('a', 1342), ('the', 1211), ('and', 868), ('in', 834), ('i', 826), ('is', 815), ('u', 803), ('me', 749), ('for', 656), ('my', 638), ('of', 598), ('it', 564), ('your', 562), ('on', 504), ('have', 481), ('that', 468), ('now', 404)]
20 top spam words: [('to', 611), ('a', 360), ('call', 188), ('you', 187), ('your', 187), ('or', 185), ('the', 178), ('for', 170), ('is', 149), ('on', 139), ('Call', 138), ('now', 138), ('have', 128), ('and', 119), ('FREE', 116), ('from', 116), ('ur', 107), ('with', 102), ('www', 96), ('mobile', 95)]
20 top ham words: [('you', 1645), ('to', 1548), ('I', 1490), ('the', 1033), ('a', 982), ('i', 819), ('in', 766), ('and', 749), ('u', 735), ('me', 720), ('is', 666), ('my', 628), ('it', 540), ('of', 505), ('for', 486), ('that', 445), ('your', 375), ('on', 365), ('not', 355), ('have', 353)]


**Ответ**:

20 top total words:

    [('to', 2159), ('you', 1832), ('I', 1533), ('a', 1342), ('the', 1211), ('and', 868), ('in', 834), ('i', 826), ('is', 815), ('u', 803), ('me', 749), ('for', 656), ('my', 638), ('of', 598), ('it', 564), ('your', 562), ('on', 504), ('have', 481), ('that', 468), ('now', 404)]

20 top spam words:

    [('to', 611), ('a', 360), ('call', 188), ('you', 187), ('your', 187), ('or', 185), ('the', 178), ('for', 170), ('is', 149), ('on', 139), ('Call', 138), ('now', 138), ('have', 128), ('and', 119), ('FREE', 116), ('from', 116), ('ur', 107), ('with', 102), ('www', 96), ('mobile', 95)]

20 top ham words:

    [('you', 1645), ('to', 1548), ('I', 1490), ('the', 1033), ('a', 982), ('i', 819), ('in', 766), ('and', 749), ('u', 735), ('me', 720), ('is', 666), ('my', 628), ('it', 540), ('of', 505), ('for', 486), ('that', 445), ('your', 375), ('on', 365), ('not', 355), ('have', 353)]


4. Определите для каждой из категорий, сколько сообщений содержат хотя бы одно из 20 популярных слов для этой же
и другой категории.

In [14]:
# Словарь для хранения числа файлов, содержащих топ-20:
#       первый элемент в списке - слова из топ-20 спама,
#       второй - слова из топ-20 обычных.
count_of_top_20 = {
    'spam': [0, 0],
    'ham': [0, 0]
}

for _, subdir in enumerate(subdirs):
    subdir_path = os.path.join(BASE_DATA_DIR, subdir)
    files = os.listdir(subdir_path)

    for _, file in enumerate(files):
        # Флаги, которые принимают значение тру, если файл содержит необходимое слово
        # в соответсвующем словаре
        contain_spam = False
        contain_ham = False

        file_path = os.path.join(subdir_path, file)
        with open(file_path, 'r', errors='ignore') as f:
            words = re.findall(r"\b[A-Za-z-']+\b", f.readline())

            for word in words:
                if word in dict(sorted_spam[:20]).keys():
                    contain_spam = True
                if word in dict(sorted_ham[:20]).keys():
                    contain_ham = True

        # Конвертируем флаги в int(), то есть True -> 1, False -> 0,
        # и обновляем соответствующее значение списка по соответсвующему ключу
        count_of_top_20[subdir][0] += int(contain_spam)
        count_of_top_20[subdir][1] += int(contain_ham)

# Вывод информации
print(f"Amount of spam message that:\n\tcontain top-20 spam: {count_of_top_20['spam'][0]}\n"
      f"\tcontain top-20 ham: {count_of_top_20['spam'][1]}\n"
      f"Amount of ham message that:\n\tcontain top-20 spam: {count_of_top_20['ham'][0]}\n"
      f"\tcontain top-20 ham: {count_of_top_20['ham'][1]}\n")

Amount of spam message that:
	contain top-20 spam: 730
	contain top-20 ham: 698
Amount of ham message that:
	contain top-20 spam: 3501
	contain top-20 ham: 4087



**Ответ**:

Amount of *spam* message that:

	contain top-20 spam: 730
	contain top-20 ham: 698
Amount of *ham* message that:

	contain top-20 spam: 3501
	contain top-20 ham: 4087

5. Определите для каждой из категорий, сколько сообщений содержат хотя бы одно из 10 популярных слов для этой же и другой категории.


In [15]:
# Аналогично предыдущему, только срез в топе делаем по 10-ти топ словам

count_of_top_10 = {
    'spam': [0, 0],
    'ham': [0, 0]
}

for _, subdir in enumerate(subdirs):
    subdir_path = os.path.join(BASE_DATA_DIR, subdir)
    files = os.listdir(subdir_path)

    for _, file in enumerate(files):
        contain_spam = False
        contain_ham = False
        file_path = os.path.join(subdir_path, file)
        with open(file_path, 'r', errors='ignore') as f:
            words = re.findall(r"\b[A-Za-z-']+\b", f.readline())
            for word in words:
                if word in dict(sorted_spam[:10]).keys():
                    contain_spam = True
                if word in dict(sorted_ham[:10]).keys():
                    contain_ham = True
        count_of_top_10[subdir][0] += int(contain_spam)
        count_of_top_10[subdir][1] += int(contain_ham)

print(f"Amount of spam message that:\n\tcontain top-10 spam: {count_of_top_10['spam'][0]}\n"
      f"\tcontain top-10 ham: {count_of_top_10['spam'][1]}\n"
      f"Amount of ham message that:\n\tcontain top-10 spam: {count_of_top_10['ham'][0]}\n"
      f"\tcontain top-10 ham: {count_of_top_10['ham'][1]}\n")


Amount of spam message that:
	contain top-10 spam: 705
	contain top-10 ham: 632
Amount of ham message that:
	contain top-10 spam: 3190
	contain top-10 ham: 3720



**Ответ**:

Amount of spam message that:

	contain top-10 spam: 705
	contain top-10 ham: 632
Amount of ham message that:

	contain top-10 spam: 3190
	contain top-10 ham: 3720



6. Найдите 20 самых длинных слов по всем сообщениям.

In [16]:
# Просто в общем словаре делаем сортировку по длине ключа, то есть слова
sorted_by_length = sorted(dictionary['total'].items(), key=lambda x: len(x[0]), reverse=True)

# Вывод первого элемента из пары значений, то есть только слова
print(f"Top-10 the longest words: {[value[0] for value in sorted_by_length[:10]]}")

Top-10 the longest words: ['hypotheticalhuagauahahuagahyuhagga', 'AccommodationVouchers', 'PRESCHOOLCO-ORDINATOR', 'MonthlySubscription', 'XXXMobileMovieClub', 'xxxmobilemovieclub', 'miiiiiiissssssssss', 'friend-of-a-friend', 'HAUGHAIGHGTUJHYGUJ', 'DeliveredTomorrow']


**Ответ**:

Top-10 the longest words:

    ['hypotheticalhuagauahahuagahyuhagga', 'AccommodationVouchers', 'PRESCHOOLCO-ORDINATOR', 'MonthlySubscription', 'XXXMobileMovieClub', 'xxxmobilemovieclub', 'miiiiiiissssssssss', 'friend-of-a-friend', 'HAUGHAIGHGTUJHYGUJ', 'DeliveredTomorrow']
